# Use examples of [premise](https://github.com/romainsacchi/premise) with user-generated scenarios

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to adapt the life cycle inventory database [ecoinvent](https://www.ecoinvent.org/) for prospective environmental impact assessment, using **user-generated scenarios**.

User-generated scenario are scenarios built by the premise users community. They can be used on their own
or together with a global IAM scenario.
Public community scenarios are listed under the repository [Premise community scenario](https://github.com/premise-community-scenarios).

User-generated scenarios are "packaged" into [data packages](https://specs.frictionlessdata.io/data-package/).

Data packages ensure that the data used comes with all necessary metadata, resources (scenario data, inventories, etc.), and that the data is formatted in a correct way.

To fetch a data packge, you can use the `datapackage` library.
For exmaple, let's fetch the user-generated scenario about [Switzerland's future energy supply](https://github.com/premise-community-scenarios/energy-perspective-2050-switzerland).

In [1]:
from premise import *

clear_cache()

Cache folder cleared!


In [2]:
from datapackage import Package

In [3]:
# URL pointing to the raw datapackage.json file of the scenario (on github or computer)
#url = r"https://raw.githubusercontent.com/alideoro/Team_PV/main/energy-perspective-2050-switzerland-main2/datapackage.json"
url = r"https://raw.githubusercontent.com/premise-community-scenarios/energy-perspective-2050-switzerland/main/datapackage.json"

switzerland_2050 = Package(url)

The datapackage has 3 resources: scenario data, inventories and a configuration file.

In [4]:
for resource in switzerland_2050.resources:
    print(resource.name)

scenario_data
inventories
config


or

In [5]:
switzerland_2050.resource_names

['scenario_data', 'inventories', 'config']

And you can directly read them (or look the resources up directly from the repo):

In [6]:
import pandas as pd

In [7]:
#data.raw_read()

data2 = switzerland_2050.get_resource("scenario_data").read()
df=pd.DataFrame(data2)

In [8]:
df.loc[:, 4].unique()

array(['Production|Electricity|Run-of-river hydro',
       'Production|Electricity|Reservoir',
       'Production|Electricity|Nuclear|Boiling water reactor',
       'Production|Electricity|Nuclear|Pressure water reactor',
       'Production|Electricity|Conventional|Waste-to-Energy',
       'Production|Electricity|Conventional|Other',
       'Production|Electricity|Renewable|Photovoltaic',
       'Production|Electricity|Renewable|Wind turbines|Onshore',
       'Production|Electricity|Renewable|Geothermal',
       'Production|Electricity|Renewable|Biomass',
       'Production|Electricity|Renewable|Biogas',
       'Production|Electricity|Medium to high',
       'Production|Electricity|Low to medium',
       'Production|Electricity|Import',
       'Transmission losses|Electricity|Medium to high',
       'Distribution losses|Electricity|Low to medium',
       'Production|Liquid fuel|Diesel|Diesel',
       'Production|Liquid fuel|Diesel|Biodiesel',
       'Production|Liquid fuel|Diesel|Synth

The datapackage can also be locally stored (i.e., on your computer), in which case you only
need to provide the filepath to the `datapackage.json` file.

Once the datapackage is loaded, you just need to pass it (in a list) to the `NewDatabase` instance of `premise`.

In [9]:
from premise import *
import bw2data
bw2data.projects.set_current("ei38-teaching")
bw2data.databases

Databases dictionary with 8 object(s):
	Final_IAM_User_30
	Final_IAM_User_40
	Final_IAM_User_50
	TeamPV
	TeamPV5
	TeamPV_with_IAM_elec
	biosphere3
	ei 3.8 cutoff

In [22]:
scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
    {"model": "image", "pathway":"SSP2-Base", "year": 2040},             # multiple scenarios can be specified
    {"model": "image", "pathway":"SSP2-Base", "year": 2050},
]

ndb2 = NewDatabase(
        scenarios = scenarios,        
        source_db="ei 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            switzerland_2050,
        ]
)


premise v.(1, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

To integrate the projections of the user-generated scenario, call `update_external_scenario()`.

In [ ]:
ndb2.update_electricity()
ndb2.update_external_scenario()


/////////////////////////// ELECTRICITY ////////////////////////////
Update natural gas extraction datasets.
Update efficiency of solar PV.
Log of changes in photovoltaics efficiencies saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Create biomass markets.
Empty old electricity datasets
Create high voltage markets.
Create medium voltage markets.
Create low voltage markets.
Log of deleted electricity markets saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Log of created electricity markets saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Done!
Adjust efficiency of power plants...
Log of changes in power plants efficiencies saved in /opt/tljh/user/envs/bw/lib/python3.10/site-packages/premise/data/logs
Rescale inventories and emissions for Biomass CHP CCS
Rescale inventories and emissions for Oil CC CCS
Rescale inventories and emissions for Oil CHP
Rescale inventories and emissions for Biomass IGC

You can combine the user-generated scenario with any, all or none of the projections for the IAM model.
For example, here with the electricity projections of the IMAGE SSP2-Base scenario:

In [1]:
temp_db = ndb.scenarios[0]["database"]

for act in temp_db:
    if "EP2050" in act["name"] and "market for electricity" in act["name"]:
        for exc in act["exchanges"]:
            if exc["type"] == "technosphere":
                print(act["name"], act["location"], exc["name"], exc["amount"], exc["location"])
                


NameError: name 'ndb' is not defined

In [ ]:

scenarios = [
    {"model": "image", "pathway":"SSP2-Base", "year": 2030},
]

ndb = NewDatabase(
        scenarios = scenarios,        
        source_db="ei 3.8 cutoff",
        source_version="3.8",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        external_scenarios=[
            switzerland_2050,
        ]
)

ndb.update_electricity()
ndb.update_external_scenario()

Once the projections are integrated, you can export the database(s) back to your Brightway2 project, to
a CSV Simapro file, or as a set of sparse amtrices (see main exmaples notebook for more details).

In [21]:
from brightway2 import *
# del databases['TeamPV4']
databases
bi.export.write_lci_csv("Final_IAM_User_50")

'/home/jupyter-jnoehl/.local/share/Brightway3/ei38-teaching.9d24c3b84b374fda3985a161a6e71437/output/lci-Final_IAM_User_50.csv'

In [16]:
ndb2.write_db_to_brightway(name=["Final_IAM_User_30", "Final_IAM_User_40", "Final_IAM_User_50"])

Write new database(s) to Brightway2.
Prepare database 1.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24036 datasets
681973 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 19:47:34
  Finished: 10/27/2022 19:48:02
  Total time elapsed: 00:00:27
  CPU %: 96.10
  Memory %: 3.99
Created database: Final_IAM_User_30
Prepare database 2.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24036 datasets
682133 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 10/27/2022 19:51:10
  Finished: 10/27/2022 19:51:40
  Total time elapsed: 00:00:29
  CPU %: 96.60
  Memory %: 3.99
Created database: Final_IAM_User_40
Prepare database 3.
- check for duplicates...
- check for values format...
- relinking exchanges...
Done!
24036 datasets
682062 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 database:
  Started: 1

In [15]:

test=Database("TeamPV_with_IAM_elec")

In [16]:
actList2 = [X for X in test if 'photovoltaic panel, PK/single-Si tandem, at regional storage' in X['name']]
testing=actList2[0]
testing.key

('TeamPV_with_IAM_elec', 'f06fa1d3c529457485a26db952483abc')

In [ ]:
#testing = [act for act in temp_db if "electricity production, photovoltaic, PK/single-Si tandem" in act["name"]][0]
#import bw2analyzer as ba
#import bw2data as bd
#bd.get_activity?
#testing = bd.get_activity(key?)
#print(testing)
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
ba.print_recursive_supply_chain(testing, amount=1, max_level=6, cutoff=0.001)

In [42]:
len(test)


20877

In [43]:
actList = [X for X in test if 'market for electricity' in X['name']                # Iterates trough activities X in database db and adds them to list 'act_list' if they contain 'text1' in 'name',
                        and 'CH' in X['location']                # 'text2' in 'name',
                        and 'low voltage' in X['name']]            # but not 'text3' in 'name',

In [73]:
actList

['market for electricity, low voltage, renewable energy products' (kilowatt hour, CH, None),
 'market for electricity, low voltage' (kilowatt hour, CH, None),
 'market for electricity, low voltage, EP2050' (kilowatt hour, CH, None)]

In [ ]:
EP2050=actList[2]

In [ ]:
excList = [exc for exc in EP2050.technosphere()]
excList

In [17]:
import brightway2 as bw
funUnit = {testing:1}  
testing

'photovoltaic panel, PK/single-Si tandem, at regional storage' (square meter, DE, None)

In [113]:
mh=methods.random()
mh

('EDIP w/o LT (superseded)',
 'resource consumption w/o LT',
 'non-renewable resources, zinc w/o LT')

In [18]:
lca=bw.LCA(funUnit, ipcc)
lca.demand

NameError: name 'ipcc' is not defined

In [122]:
lca.lci()

In [123]:
lca.lcia()

In [126]:
lca.score

147.4712327446954

In [19]:
import bw2analyzer as ba
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
# ba.print_recursive_supply_chain(testing, amount=1, max_level=6, cutoff=0.01)
ba.print_recursive_calculation(testing, ipcc, amount=1, cutoff=0.01)

Fraction of score | Absolute score | Amount | Activity
0001 | 136.1 |     1 | 'photovoltaic panel, PK/single-Si tandem, at regional storage' (square
  0.11 | 14.99 |  2.13 | 'aluminium alloy production, AlMg3' (kilogram, RER, None)
    0.0137 | 1.871 | 0.06496 | 'market for magnesium' (kilogram, GLO, None)
      0.0121 | 1.644 | 0.05381 | 'magnesium production, pidgeon process' (kilogram, CN, None)
    0.0856 | 11.65 | 2.055 | 'market for aluminium, cast alloy' (kilogram, GLO, None)
      0.0764 |  10.4 | 0.5373 | 'aluminium ingot, primary, to aluminium, cast alloy market' (kilogram,
  0.0178 | 2.417 | 0.975 | 'ethylvinylacetate production, foil' (kilogram, RER, None)
    0.0152 | 2.067 | 0.9945 | 'market for ethylene vinyl acetate copolymer' (kilogram, RER, None)
      0.015 | 2.042 | 0.9945 | 'ethylene vinyl acetate copolymer production' (kilogram, RER, None)
  0.0183 | 2.495 | 0.295 | 'glass fibre reinforced plastic production, polyamide, injection mould
    0.0164 | 2.232 | 0.3097 

In [128]:
actList2 = [X for X in test if 'photovoltaic slanted-roof installation' in X['name']]
PanelOld=actList2[8]
PanelOld

'photovoltaic slanted-roof installation, 3kWp, CIS, panel, mounted, on roof' (unit, RoW, None)

In [129]:
import bw2analyzer as ba


ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')
# ba.print_recursive_supply_chain(testing, amount=1, max_level=6, cutoff=0.01)
ba.print_recursive_calculation(PanelOld, ipcc, amount=1, cutoff=0.01)

Fraction of score | Absolute score | Amount | Activity
0001 |  5580 |     1 | 'photovoltaic slanted-roof installation, 3kWp, CIS, panel, mounted, on
  0.632 |  3528 | 28.92 | 'market for photovoltaic panel, CIS' (square meter, GLO, None)
    0.227 |  1268 | 10.41 | 'photovoltaic panel production, CIS' (square meter, DE, None)
      0.203 |  1131 | 10.41 | 'market for photovoltaic laminate, CIS' (square meter, GLO, None)
      0.0223 | 124.3 | 16.35 | 'market for aluminium alloy, AlMg3' (kilogram, GLO, None)
    0.405 |  2261 | 18.51 | 'photovoltaic panel production, CIS' (square meter, RoW, None)
      0.0396 |   221 | 29.06 | 'market for aluminium alloy, AlMg3' (kilogram, GLO, None)
      0.36 |  2011 | 18.51 | 'market for photovoltaic laminate, CIS' (square meter, GLO, None)
  0.101 | 562.2 |   2.4 | 'market for inverter, 2.5kW' (unit, GLO, None)
    0.0325 | 181.2 | 0.7934 | 'inverter production, 2.5kW' (unit, RER, None)
    0.0683 | 381.1 | 1.607 | 'inverter production, 2.5kW' (uni